In [12]:
import numpy as np
import cocoex
from skopt import gp_minimize 
from tqdm import tqdm
from pak.function import *
from numpy.random import standard_normal as _randn # TODO: may bring confusion
import warnings
warnings.filterwarnings("ignore")

In [13]:
# 创建测试套件并获取问题实例
f = [F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24]
bounds = [(-5., 5.), ] * 2

In [14]:
n_repeats = 1
n_calls = 100
n_init = 10
noise_level =0.
id = 0

In [15]:
def fnoise(x):
    y = f[id](x) + noise_level * std[id] * _randn()
    return y 

In [16]:
import csv
for i in range(5,6,1):
    id = i
    results = []
    noimin = []
    obsmin = []
    premin = []
    with tqdm(total=n_repeats*n_calls,desc = "Optimizing") as pbar:
        def callback(result):
            if len(result.models)==0:
                noimin.append(result.x_iters[-1])
                obsmin.append(result.x_iters[-1])
                premin.append(result.x_iters[-1])   
            else:
                noimin.append(result.x)
                x_iters = np.array(result.x_iters)
                y_iters = np.array(result.func_vals)
                md = result.models[-1].fit(x_iters,y_iters)
                curmin = 1000000000
                for obs in x_iters:
                    pd = md.predict(obs.reshape(1,-1))
                    if pd < curmin:
                        temp = obs
                        curmin = pd
                obsmin.append(temp.tolist())
                res = gp_minimize(fnoise,bounds,n_initial_points=0,acq_func="LCB",n_calls=1,random_state=1,kappa=0,x0=result.x_iters,y0=result.func_vals)
                premin.append(res.x_iters[-1])
            pbar.update(1)
            
        results = [gp_minimize(fnoise,bounds,n_initial_points=n_init,initial_point_generator='lhs',acq_func="EI",n_calls=n_calls,random_state=n,callback=callback)
                for n in range(n_repeats)]
'''      
    for result in results:
        with open("Fn"+str(i)+"M.csv","a") as file:
            np.savetxt(file,result.x_iters,delimiter=',')
    
    with open("Fn"+str(i)+"M_noi.csv","a") as file:
        np.savetxt(file,noimin,delimiter=',')

    with open("Fn"+str(i)+"M_obs.csv","a") as file:
        np.savetxt(file,obsmin,delimiter=',')

    with open("Fn"+str(i)+"M_pre.csv","a") as file:
        np.savetxt(file,premin,delimiter=',')
'''

Optimizing: 100%|██████████| 100/100 [00:57<00:00,  1.74it/s]


'      \n    for result in results:\n        with open("Fn"+str(i)+"M.csv","a") as file:\n            np.savetxt(file,result.x_iters,delimiter=\',\')\n    \n    with open("Fn"+str(i)+"M_noi.csv","a") as file:\n        np.savetxt(file,noimin,delimiter=\',\')\n\n    with open("Fn"+str(i)+"M_obs.csv","a") as file:\n        np.savetxt(file,obsmin,delimiter=\',\')\n\n    with open("Fn"+str(i)+"M_pre.csv","a") as file:\n        np.savetxt(file,premin,delimiter=\',\')\n'